In [1]:
!ngrok kill
#!pip uninstall nltk
!pip install flask
!pip install flask-cors
!pip install transformers
!pip install wikipedia-api
!pip install langdetect
!pip install nltk
!pip install spacy
!python -m spacy download pt_core_news_sm
!pip install keras
!pip install torch

from flask import Flask, request, jsonify
from flask_cors import CORS
import threading
import wikipediaapi
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer, SnowballStemmer, LancasterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tag import pos_tag, pos_tag_sents
import string
import time
from langdetect import detect
from nltk.corpus import wordnet
from os.path import join
import spacy
import numpy as np
import json
import pandas as pd
from collections import deque
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score , recall_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.utils import to_categorical
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from transformers import DebertaV2ForQuestionAnswering, DebertaV2Tokenizer
import torch
from sklearn.tree import DecisionTreeClassifier

nltk.download("stopwords")
stopwords_pt = stopwords.words('portuguese')
nltk.download("punkt_tab")
nltk.download("tagsets")
nltk.download("wordnet")
nltk.download("averaged_perceptron_tagger")
nltk.download("maxent_ne_chunker")
nltk.download("words")
nltk.download('tagsets_json')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker_tab')
nltk.download('omw-1.4')

nlp = spacy.load("pt_core_news_sm")

wiki_wiki = wikipediaapi.Wikipedia(
    language='pt',
    extract_format=wikipediaapi.ExtractFormat.WIKI,
    user_agent="ProjetoIA/1.0 (http://localhost:3000/projeto-ia) Chat"
)

app = Flask(__name__)
CORS(app)

def iniciar(caminho_arquvio, rota, texto):
  def buscar_jogos_categoria(limite_total=2000):
    with open('minha_lista_jogos.json', 'r') as file:
        data = json.load(file)

    def extrair_strings(obj):
        result = []
        if isinstance(obj, dict):
            for value in obj.values():
                result.extend(extrair_strings(value))
        elif isinstance(obj, list):
            for item in obj:
                result.extend(extrair_strings(item))
        elif isinstance(obj, str):
            result.append(obj)
        return result

    todas_as_strings = extrair_strings(data)

    return todas_as_strings

  jogos_totais = buscar_jogos_categoria(limite_total=2500)

  print(f"Total de jogos encontrados: {len(jogos_totais)}")
  for jogo in list(jogos_totais)[:100]:
      print(jogo)

  with open(caminho_arquvio, 'r') as file:
      data = json.load(file)

  perguntas = [item['pergunta'] for item in data]
  respostas = [item['resposta'] for item in data]
  categorias = [item['classe'] for item in data]

  vectorizer = TfidfVectorizer(max_features=500, stop_words=stopwords_pt)
  X = vectorizer.fit_transform(perguntas)
  labelencoder = LabelEncoder()
  y = labelencoder.fit_transform(categorias)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  clf = DecisionTreeClassifier(max_depth=10, random_state=42)
  clf.fit(X_train, y_train)

  print("Categorias conhecidas pelo LabelEncoder:", labelencoder.classes_)
  y_pred = clf.predict(X_test)
  print("Valores previstos pelo modelo:", y_pred)
  print("Número de categorias conhecidas:", len(labelencoder.classes_))

  def processar_pergunta(texto):
    sentencas = sent_tokenize(texto, language = "portuguese")
    print(sentencas)

    palavras = word_tokenize(texto, language = "portuguese")
    print(palavras)

    stops = stopwords.words("portuguese")
    print(stops)

    palavras_sem_stopwords = [p for p in palavras if p not in stops]
    print(palavras_sem_stopwords)

    palavras_sem_pontuacao = [p for p in palavras_sem_stopwords if p not in string.punctuation]
    print(palavras_sem_pontuacao)

    frequencia = nltk.FreqDist(palavras_sem_pontuacao)
    print(frequencia)
    print(frequencia.items())

    mais_comuns = frequencia.most_common(2)
    print(mais_comuns)

    stemmer = PorterStemmer()
    stem1 = [ stemmer.stem(word) for word in palavras_sem_pontuacao]
    print(stem1)

    nltk.help.upenn_tagset()

    pos = nltk.pos_tag(palavras_sem_pontuacao)
    print(pos)

    en = nltk.ne_chunk(pos)
    print(en)

    labelencoder = LabelEncoder()
    mess = labelencoder.fit_transform(palavras_sem_pontuacao)
    print(mess)

    def encontrar_jogo_consecutivo(palavras, lista_jogos):
        jogos_encontrados = []
        i = 0
        while i < len(palavras):
            for jogo in lista_jogos:
                jogo_palavras = jogo.lower().split()

                if len(jogo_palavras) > len(palavras) - i:
                    continue

                match = True
                for j in range(len(jogo_palavras)):
                    if palavras[i + j].lower() != jogo_palavras[j]:
                        match = False
                        break

                if match:
                    jogos_encontrados.append(jogo)
                    i += len(jogo_palavras)
                    break
            else:
                i += 1

        return jogos_encontrados

    jogos_detectados_consecutivos = encontrar_jogo_consecutivo(palavras_sem_pontuacao, jogos_totais)
    #jogos_detectados_consecutivos = encontrar_jogo_consecutivo(palavras_sem_pontuacao, ["Life is strange"])
    print("Jogos detectados na frase:", jogos_detectados_consecutivos)

    def detectar_questao_principal(pos, jogos_encontrados):
        NNs = []
        jogos_encontrados_lower = [jogo.lower() for jogo in jogos_encontrados]
        jogos_palavras = [jogo.split() for jogo in jogos_encontrados_lower]

        for i in pos:
            if i[1] == 'NN' or i[1] == 'NNS' or i[1] == 'NNP' or i[1] == 'NNPS' or i[1] == 'JJ':
                palavra_lower = i[0].lower()

                palavra_encontrada = False
                for jogo_palavras in jogos_palavras:
                    # if palavra_lower in jogo_palavras and palavra_lower != "artista":
                    if palavra_lower in jogo_palavras:
                        palavra_encontrada = True
                        break

                if not palavra_encontrada:
                    NNs.append(i)

        lemmatizer = WordNetLemmatizer()
        stemmer2 = PorterStemmer()

        NNs_stemmed = []

        for word, tag in NNs:
            language = detect(word)

            if language == 'en':
                processed_word = lemmatizer.lemmatize(word)
            else:
                processed_word = stemmer2.stem(word)

            NNs_stemmed.append((processed_word, tag))

        seen = set()
        NNs_stemmed_unique = []
        for word, tag in NNs_stemmed:
            if (word, tag) not in seen:
                NNs_stemmed_unique.append((word, tag))
                seen.add((word, tag))

        print("NNs após stemmer e removendo duplicatas:", NNs_stemmed_unique)
        return NNs_stemmed_unique

    NNs_stemmed_unique_global = detectar_questao_principal(pos, jogos_detectados_consecutivos)

    pagina = wiki_wiki.page(''.join(jogos_detectados_consecutivos[0]).replace(" ", "_"))

    sentencas_jogo = sent_tokenize(pagina.text, language = "portuguese")
    print(sentencas_jogo)
    print(NNs_stemmed_unique_global)

    palavras_stemmed = []

    for tupla in NNs_stemmed_unique_global:
        palavra = tupla[0]

        if palavra.lower() != 'jogo' and palavra.lower() != 'gosto':
          palavras_stemmed.append(palavra.lower())

    print("Texto processado:", ' '.join(palavras_sem_pontuacao))
    input_vector = vectorizer.transform([' '.join(palavras_sem_pontuacao)])
    categoria_predita = clf.predict(input_vector)[0]
    print(categoria_predita)

    if categoria_predita < len(labelencoder.classes_):
      categoria_nome = labelencoder.inverse_transform([categoria_predita])[0]
    else:
      categoria_nome = "Desconhecido"

    sentencas_filtradas = []

    for sentenca in sentencas_jogo:
        sentenca_lower = sentenca.lower()

        for palavra in palavras_stemmed:
            if palavra in sentenca_lower:
                sentencas_filtradas.append(sentenca)

    print(sentencas_filtradas)

    def prever_categoria(pergunta):
        input_vector = vectorizer.transform([pergunta])
        categoria_predita = clf.predict(input_vector)[0]
        probabilidade = max(clf.predict_proba(input_vector)[0])

        if categoria_predita < len(labelencoder.classes_):
          categoria_nome = labelencoder.inverse_transform([categoria_predita])[0]
        else:
          categoria_nome = "Desconhecido"

        return categoria_nome

    pergunta_usuario = ' '.join(palavras_sem_pontuacao)
    categoria_predita = prever_categoria(pergunta_usuario)
    print(f"Categoria predita: {categoria_predita}")

    def encontrar_melhor_sentenca(sentencas_filtradas, categoria):
        sentencas_vetorizadas = vectorizer.transform(sentencas_filtradas)
        pergunta_vetorizada = vectorizer.transform([pergunta_usuario])
        similaridades = cosine_similarity(pergunta_vetorizada, sentencas_vetorizadas)

        melhor_indice = np.argmax(similaridades)
        return sentencas_filtradas[melhor_indice]

    melhor_sentenca = encontrar_melhor_sentenca(sentencas_filtradas, categoria_predita)
    print(f"Melhor sentença: {melhor_sentenca}")

    return f'{melhor_sentenca}'

  response = processar_pergunta(texto)
  return response

@app.route('/send_data', methods=['POST'])
def receive_data():
    data = request.get_json()
    message = data.get('message')
    texto = message
    melhor_sentenca_response = iniciar('jogos_respostas_com_classe.json', 'send_data', texto)
    response = {'reply': melhor_sentenca_response}
    return jsonify(response)
# iniciar(["Séries_de_televisão"], 'series_perguntas_respostas.json', 'send_serie')

def run_app():
    app.run(port=5002)

thread = threading.Thread(target=run_app)
thread.start()

!pip install pyngrok
from pyngrok import ngrok

!ngrok config add-authtoken 2pLbbmL9tFk7XrAPfN53Ubba52f_7V47PyqU95adT2vGs98Nr

public_url = ngrok.connect(5002)
print('Link público:', public_url)

ngrok - tunnel local ports to public URLs and inspect traffic

USAGE:
  ngrok [command] [flags]

AUTHOR:
  ngrok - <support@ngrok.com>

COMMANDS: 
  config          update or migrate ngrok's configuration file
  http            start an HTTP tunnel
  tcp             start a TCP tunnel
  tunnel          start a tunnel for use with a tunnel-group backend

EXAMPLES: 
  ngrok http 80                                                 # secure public URL for port 80 web server
  ngrok http --url baz.ngrok.dev 8080                           # port 8080 available at baz.ngrok.dev
  ngrok tcp 22                                                  # tunnel arbitrary TCP traffic to port 22
  ngrok http 80 --oauth=google --oauth-allow-email=foo@foo.com  # secure your app with oauth

Paid Features: 
  ngrok http 80 --url mydomain.com                              # run ngrok with your own custom domain
  ngrok http 80 --cidr-allow 2600:8c00::a03c:91ee:fe69:9695/32  # run ngrok with IP policy restrictions

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package tagsets_json to /root/nltk_data...
[nltk_data]   P

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5002
INFO:werkzeug:Press CTRL+C to quit


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Link público: NgrokTunnel: "https://e63b-34-23-194-47.ngrok-free.app" -> "http://localhost:5002"
